<a href="https://colab.research.google.com/github/NancyYiWang/WildFireSmokePrediction/blob/main/PredictModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install rasterio

In [8]:
# deal with terrain data

import rasterio
import numpy as np

with rasterio.open("/content/drive/My Drive/WildFire/DATA/Terrain/terrain_broad_calgary.tiff") as src:

    band_count = src.count
    print(f"Number of bands in the TIFF file: {band_count}")

    bands = [src.read(band) for band in range(1, band_count + 1)]
    transform = src.transform

bands_array = np.array(bands)

print(f"Shape of bands_array: {bands_array.shape}")
print(f"Transform: {transform}")

Number of bands in the TIFF file: 1
Shape of bands_array: (1, 720, 960)
Transform: | 0.00, 0.00,-116.00|
| 0.00,-0.00, 52.50|
| 0.00, 0.00, 1.00|


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

def load_nc_data(file_path):
    data = xr.open_dataset(file_path)
    return data.to_dataframe().reset_index()

def preprocess_data(weather_data, terrain_data):

    merged_data = weather_data.merge(terrain_data, on=["latitude", "longitude"])
    return merged_data

def train_fire_prediction_model(data):

    X = data[['uwnd', 'vwnd', 'rhum', 'air', 'hgt']]
    y = data['fire_occurred']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of the fire prediction model: {accuracy}")
    return model

def simulate_smoke_dispersion(data, fire_model):

    data['fire_probability'] = fire_model.predict_proba(data[['uwnd', 'vwnd', 'rhum', 'air', 'hgt']])[:, 1]

    def gaussian_dispersion(fire_prob, wind_speed, distance):
        return fire_prob * np.exp(-distance / (2 * wind_speed))

    data['wind_speed'] = np.sqrt(data['uwnd']**2 + data['vwnd']**2)
    data['smoke_concentration'] = gaussian_dispersion(
        data['fire_probability'], data['wind_speed'], distance=1
    )
    return data

def train_smoke_prediction_system(weather_data, terrain_data, fire_labels):

    data = preprocess_data(weather_data, terrain_data)
    data['fire_occurred'] = fire_labels

    fire_model = train_fire_prediction_model(data)

    final_data = simulate_smoke_dispersion(data, fire_model)

    X = final_data[['fire_probability', 'wind_speed', 'rhum', 'air', 'hgt']]
    y = final_data['smoke_concentration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    smoke_model = RandomForestRegressor(n_estimators=100, random_state=42)
    smoke_model.fit(X_train, y_train)

    y_pred = smoke_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"MSE of the smoke prediction model: {mse}")
    return smoke_model

def predict_real_time(weather_data, terrain_data, smoke_model):

    data = preprocess_data(weather_data, terrain_data)
    predictions = smoke_model.predict(data[['fire_probability', 'wind_speed', 'rhum', 'air', 'hgt']])
    data['predicted_smoke_concentration'] = predictions
    return data

uwnd_data = load_nc_data("/content/drive/My Drive/WildFire/DATA/NOAA_Climate/uwnd.2024.nc")
vwnd_data = load_nc_data("/content/drive/My Drive/WildFire/DATA/NOAA_Climate/vwnd.2024.nc")
rhum_data = load_nc_data("/content/drive/My Drive/WildFire/DATA/NOAA_Climate/rhum.2024.nc")
air_data = load_nc_data("/content/drive/My Drive/WildFire/DATA/NOAA_Climate/air.2024.nc")
hgt_data = load_nc_data("/content/drive/My Drive/WildFire/DATA/NOAA_Climate/hgt.2024.nc")

fire_labels = pd.read_csv("fire_labels.csv")['fire_occurred']

smoke_model = train_smoke_prediction_system(weather_data, terrain_data, fire_labels)

real_time_data = predict_real_time(weather_data, terrain_data, smoke_model)